<a href="https://colab.research.google.com/github/2020-nlp-c/nlp-statisticsmodel/blob/master/BaeJinwoo/TextRank_%EA%B5%AC%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
text = '딸기 바나나 사과 파인애플 수박.바나나 사과 딸기 포도.복숭아 수박.파인애플 사과 딸기 바나나'

In [ ]:
sentence_list = text.split('.')
sentence_list

['딸기 바나나 사과 파인애플 수박', '바나나 사과 딸기 포도', '복숭아 수박', '파인애플 사과 딸기 바나나']

In [ ]:
sentence_list #node
sentence_token_list = []
for i in range(len(sentence_list)):
    sentence = sentence_list[i].split(' ') #.뒤에 띄어쓰기하는 부분 포함하면 안됌. 개선필요
    sentence_token_list.append(sentence)
sentence_token_list

[['딸기', '바나나', '사과', '파인애플', '수박'],
 ['바나나', '사과', '딸기', '포도'],
 ['복숭아', '수박'],
 ['파인애플', '사과', '딸기', '바나나']]

In [ ]:
tokens = ['딸기', '바나나', '사과', '딸기', '파인애플', '수박', '포도', '복숭아']


In [ ]:
#토큰 인덱싱
tokens2idx = {tokens[i]:i for i in range(0, len(tokens))} #tokens을 인덱스로 변환
idx2tokens = {i:tokens[i] for i in range(0, len(tokens))} #인덱스를 tokens으로 변환
tokens2idx

{'딸기': 3, '바나나': 1, '복숭아': 7, '사과': 2, '수박': 5, '파인애플': 4, '포도': 6}

In [ ]:
#2차원 배열 생성
import numpy as np
graph_edge = np.zeros((len(sentence_list),len(sentence_list)), dtype= np.float32)
graph_edge

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer( max_features=10000)


for i in range(len(sentence_list)):
    for j in range(len(sentence_list)) : #sentence_list[i]와 sentence_list[j]를 비교하는데.. 
        temp_sum = 0
        for x in range(len(sentence_token_list[i])) : #sentence_list[i][x]의 값이 sentence_list[j]에 있으면 +1
            if sentence_token_list[i][x] in sentence_list[j] :
                temp_sum += 1
                # print('{} in {}'.format(sentence_token_list[i][x], sentence_list[j]))
        #두 문장을 합쳐서 유니크한 토큰 구하기
        sum_sentence = [sentence_list[i] + ' ' + sentence_list[j]]
        word_count_vector = count_vectorizer.fit_transform(sum_sentence)
        graph_edge[i][j] = temp_sum/len(list(count_vectorizer.vocabulary_.keys()))
graph_edge
        

array([[1.        , 0.5       , 0.16666667, 0.8       ],
       [0.5       , 1.        , 0.        , 0.6       ],
       [0.16666667, 0.        , 1.        , 0.        ],
       [0.8       , 0.6       , 0.        , 1.        ]], dtype=float32)

In [ ]:
#엣지 가중치 구하기

#매트릭스 0으로 초기화
graph_edge_weight = np.zeros((len(sentence_list),len(sentence_list)), dtype= np.float32)
graph_edge_weight


array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)

In [ ]:
#graph_edge 자기자신에게 가는 곳 0으로 초기화
for i in range(len(graph_edge)) :
    graph_edge[i][i] = 0
graph_edge

array([[0.        , 0.5       , 0.16666667, 0.8       ],
       [0.5       , 0.        , 0.        , 0.6       ],
       [0.16666667, 0.        , 0.        , 0.        ],
       [0.8       , 0.6       , 0.        , 0.        ]], dtype=float32)

In [ ]:
for i in range(len(graph_edge)) :
    for j in range(len(graph_edge)) :
        graph_edge_weight[i][j] = graph_edge[i][j]/sum(graph_edge[i])
graph_edge_weight

array([[0.        , 0.3409091 , 0.11363637, 0.54545456],
       [0.45454544, 0.        , 0.        , 0.54545456],
       [1.        , 0.        , 0.        , 0.        ],
       [0.57142854, 0.42857143, 0.        , 0.        ]], dtype=float32)

In [ ]:
#최초 스코어
score = [0 for x in range(len(graph_edge))]

for i in range(len(graph_edge)) :
    temp_score = 0
    for j in range(len(graph_edge)):
        temp_score += graph_edge[j][i]
    score[i] = temp_score
score

[1.4666666835546494, 1.100000023841858, 0.1666666716337204, 1.4000000357627869]

In [ ]:
d = 0.85
prev_score = score
for i in range(len(score)) :
    score[i] = (1 - d) + d*(score[i]) 
score

[1.3966666810214519, 1.0850000202655792, 0.2916666708886624, 1.340000030398369]

In [ ]:
score_comparison = 0

while True :
    
    #스코어 graph_edge의 컬럼 더해줘서 구하기
    for i in range(len(graph_edge)) :
        temp_score = 0
        for j in range(len(graph_edge)):
            temp_score += graph_edge[j][i]
        score[i] = temp_score
    # print('이전 스코어 {}'.format(score))

    #2) 스코어 계산
    d = 0.85
    prev_score = score
    for i in range(len(score)) :
        score[i] = (1 - d) + d*(score[i]) 
    print('현재 스코어 {}\n\n'.format(score))

    #3) 계산한 스코어(2)에 가중치 행을 곱해줘서 graph_edge갱신
    for i in range(len(graph_edge_weight)) :
        for j in range(len(graph_edge_weight)) :
                graph_edge[i][j] = score[i]*graph_edge_weight[i][j]
    print(graph_edge)

    if  abs(score_comparison - score[0]) < 0.0001 :
        break
    else :
        score_comparison = score[0]



    

이전 스코어 [1.4832302629947662, 1.029265135526657, 0.16032053530216217, 1.3285921812057495]
현재 스코어 [1.4107457235455514, 1.0248753651976585, 0.28627245500683784, 1.2793033540248873]


[[0.         0.48093605 0.16031201 0.7694977 ]
 [0.4658524  0.         0.         0.55902296]
 [0.28627247 0.         0.         0.        ]
 [0.73103046 0.54827285 0.         0.        ]]
이전 스코어 [1.4831553399562836, 1.0292088985443115, 0.1603120118379593, 1.328520655632019]
현재 스코어 [1.410682038962841, 1.0248275637626647, 0.2862652100622654, 1.2792425572872164]


[[0.         0.48091432 0.16030478 0.76946294]
 [0.46583068 0.         0.         0.55899686]
 [0.28626522 0.         0.         0.        ]
 [0.7309957  0.5482468  0.         0.        ]]


In [157]:
import pandas as pd
pd.DataFrame(score, index=['문장1','문장2','문장3','문장4'])

,0
문장1,1.410682
문장2,1.024828
문장3,0.286265
문장4,1.279243
